# The plan
1. Create some new features with the original table (as found in EDAv2)    

2. Group by sender and recipient to find where users are trading abnormally high between each other  
3. Check if users send back and forth similar amounts frequently (wash trading)  
4. Compute median time difference between trades for recurring transactions    

### 1. Create some new features with the original table (as found in EDAv2)  

In [4]:
# MODULES
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import sys
import os
import sqlite3
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# SCRIPTS
sys.path.append(os.path.abspath(".."))
from scripts import conversions

In [5]:
df = pd.read_csv('../data/full_transactions.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df

,hash,sender,recipient,value,gas_used,gas_price,timestamp,value_zscore
0,0x06dc5006e9cc13418808a35b5a6fb91378eb78c236db...,0x675bB023e268dCC43F543620577bCacB73047f08,0x5dC62cea20B0E7c3607aDCc61a885Ff9369dbc60,0.000000,223837,180.411926,2025-02-14 10:16:10,-0.011788
1,0x689d1453646e621d664836655d869d7301b8c7bcd1d4...,0x1ba98272731f1Ab5cC33a02F88694C39916E26dF,0x51C72848c68a965f66FA7a88855F9f7784502a7F,0.000000,153985,81.237520,2025-02-14 10:16:10,-0.011788
2,0x3057779542c82c2ae3f1b52b6739afe82719a61ad26c...,0x5d1783ddbb657fdf6FcC2eF5B6796Ac0DdeEb6e3,0x68d3A973E7272EB388022a5C6518d9b2a2e66fBf,0.000000,223550,65.091946,2025-02-14 10:16:11,-0.011788
3,0x0c664ea5c27efb322c1dc20c2a830b97a33fbd82b027...,0x3f22f60936F4f5D1E96cEdEb44beE66d0Cd7c220,0x98C3d3183C4b8A650614ad179A1a98be0a8d6B8E,0.000000,500002,63.063619,2025-02-14 10:16:11,-0.011788
4,0x1b386bb2f08f57a5287640e09e2f3640798a58d69a3a...,0x24f7eF98522dd61D529464f67BB3ffe96eA8AFC2,0x5dC62cea20B0E7c3607aDCc61a885Ff9369dbc60,0.000000,252997,47.995919,2025-02-14 10:16:11,-0.011788
...,...,...,...,...,...,...,...,...
1000460,0x8cbf01dc0bbebe854a64ccd7e9ccee8876c95ee30cfb...,0x5698D4ec03802438Bce268b02eb3817B8C3C2b3f,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0.000000,45758,1.919426,2025-02-17 06:06:35,-0.011788
1000461,0xb0fac9da7e21836663a05e8bcfd429a64e66160f10d7...,0x5baBE600b9fCD5fB7b66c0611bF4896D967b23A1,0xf45AFF520a8Ebe59f513719fc78b164e375Dd1f4,0.007562,21000,1.917395,2025-02-17 06:06:35,-0.011718
1000462,0x488d4e454120b061f3a0c41008b90d1156737845d91a...,0x99719eacc100BD48D4dbEb96E990333a7601e5B8,0x65c55F33F9493E8702FEea89B4fA357283eD3841,0.000000,302687,1.917395,2025-02-17 06:06:35,-0.011788
1000463,0xcdda18972b9baf4310ce4be72d1ec91dc32bb2f02c6b...,0x6a0913BA185848c09250D3F40DFf29dC45530978,0xb965602E41Af7A92B3F4b48729736Aded9C94E47,0.000000,500000,1.913384,2025-02-17 06:06:35,-0.011788


In [ ]:
conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False)

query = ''' 
SELECT
    sender,
    recipient,
    COUNT(*) AS transaction_count
FROM transactions
GROUP BY sender, recipient
ORDER BY transaction_count desc

'''
results = pd.read_sql_query(query, conn)
results

,sender,recipient,transaction_count
0,0x93793Bd1f3e35a0Efd098c30e486A860A0ef7551,0x68d3A973E7272EB388022a5C6518d9b2a2e66fBf,4266
1,0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13,0x1f2F10D1C40777AE1Da742455c65828FF36Df387,3228
2,0xD2C82F2e5FA236E114A81173e375a73664610998,0xD1669Ac6044269b59Fa12c5822439F609Ca54F41,2840
3,0x07aE8551Be970cB1cCa11Dd7a11F47Ae82e70E67,0x5c7BCd6E7De5423a257D81B442095A1a6ced35C5,2727
4,0xd8c5A092B7c85FB7D28EaE914D70d1e8533BBe63,0xd01a363Ac6612749Ce94694778990BC91aAAE0ba,2290
...,...,...,...
637344,0xfff2864E1232bd8Ca47e05F6b5F88Fb2558374B4,0x6BC2ca17281fd1c1BBDea697d7Bb66EDA6684F84,1
637345,0xfff3F3984c1Db103102F422F78296d5c60FcE0f0,0xC94eBB328aC25b95DB0E0AA968371885Fa516215,1
637346,0xfff7629B279430C688a8A4C565C5302c915ab991,0x704C2FCdD51fbe2e18e55F503CC4d69ADFbDcC5c,1
637347,0xfffB98B8b8A9466a962ebB00b8b29A1f8c73CCA8,0x0Db40d28a45C7ca2F7b39e7dbd6CE781476c4F0E,1
